 # Notebook for Downloading RDA Data.

In [3]:
import sys, os
import requests

In [26]:
filelist = [ "https://data.rda.ucar.edu/d330003/rksl/2022/20220828/z_tigge_c_rksl_20220828120000_GEPS_glob_prod_etctr_nwp.xml"
 ]

Change the value of `save_dir` if you prefer your files saved somewhere other than the current directory.

 ## Now to download the files

In [27]:
org = 'rksl'+'/'
save_dir = f'/home1/jek/Pangu-Weather/input_data/TIGGE/{org}'

for file in filelist:

    if '20220828' not in file:
        continue
    # org 앞의 경로를 제거하여 상대 경로 생성
    relative_path = file.split(org, 1)[-1]
    
    # 저장할 전체 경로 구성
    save_path = os.path.join(save_dir, relative_path)
    
    # 저장할 디렉토리 생성 (없으면 생성)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    print('Downloading', file)
    
    # 파일 다운로드 및 저장
    req = requests.get(file, allow_redirects=True)
    with open(save_path, 'wb') as f:
        f.write(req.content)

    print(f'Saved to {save_path}')

Saved to /home1/jek/Pangu-Weather/input_data/TIGGE/rksl/2022/20220828/z_tigge_c_rksl_20220828120000_GEPS_glob_prod_etctr_nwp.xml


### Once you have downloaded the data, the next part can help you plot it.

In order to plot this data, you may need to install some libraries. The easiest way to do this is to use conda or pip, however any method of getting the following libraries will work.

In [ ]:
import xarray # used for reading the data.
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

In [ ]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

In [ ]:
# Now to load in the data to xarray
ds = xarray.open_dataset(selected_file.value)

In [ ]:
# Helper methods# Define function to get standard dimensions
def get_primary(dataset):
    primary_variables = {}
    coords = dataset.coords.keys()
    highest_dims = 0
    for cur_key,cur_var in dataset.variables.items():
        if cur_key not in coords:
            primary_variables[cur_key] = cur_var
    return primary_variables 

In [ ]:
var = widgets.Dropdown(
    options=get_primary(ds).keys(),
    description='Variable')
display(var)

In [ ]:
proj = ccrs.Mercator()
plt.gcf().set_size_inches(20,10)
ax = plt.axes(projection=proj)
data_slice = ds[var.value].isel(time=0)
data_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()